In [33]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

Now let's play around a bit with the large baby names dataset we saw in lecture 1. We'll start by loading that dataset from the social security administration's website.

To keep the data small enough to avoid crashing datahub, we're going to look at only California rather than looking at the national dataset.

In [34]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
139941,CA,F,1997,Selma,15
220040,CA,F,2017,Zyanna,8
338440,CA,M,2001,Brice,19
85521,CA,F,1980,Cristela,5
380031,CA,M,2016,Aidan,297


Goal 1: Find the most popular baby name in California in 2018

In [35]:
babynames[babynames["Year"] == 2018].sort_values(by = "Count", ascending = False).head(5)

,State,Sex,Year,Name,Count
221160,CA,F,2018,Emma,2743
385701,CA,M,2018,Noah,2569
221161,CA,F,2018,Mia,2499
221162,CA,F,2018,Olivia,2465
385702,CA,M,2018,Liam,2413


Goal 2: Baby names that start with j. 

In [36]:
starts_with_j = babynames["Name"].str.startswith('J')
starts_with_j.head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: Name, dtype: bool

In [37]:
pd.DataFrame(babynames[starts_with_j]["Name"].unique()).sample(10)

,0
687,Jazlin
1302,Javin
409,Janese
1307,Jhovani
1351,Joseeduardo
1234,Jesusantonio
1267,Johncarlo
659,Jewelia
1178,Jarel
885,Jahira


Goal 2: Sort names by their length.

Approach 1a: Create a new series of only the lengths. Then add that series to the dataframe as a column. Then sort by that column. Then drop that column.

In [38]:
#create a new series of only the lengths
babyname_lengths = babynames["Name"].map(len)

#add that series to the dataframe as a column
babynames["name_lengths"] = babyname_lengths

#sort by that column
babynames_by_length = babynames.sort_values(by = "name_lengths")

#drop that column
babynames_by_length = babynames_by_length.drop("name_lengths", 1)
babynames_by_length.head(5)

,State,Sex,Year,Name,Count
85197,CA,F,1980,Ai,6
95798,CA,F,1984,Ly,8
309713,CA,M,1989,Ed,5
334020,CA,M,1999,Jr,9
339817,CA,M,2001,Ed,5


Approach 1b: Same as 1a, but use str.len() to generate the lengths of the strings.

In [39]:
#create a new series of only the lengths
babyname_lengths = babynames["Name"].str.len()

#add that series to the dataframe as a column
babynames["name_lengths"] = babyname_lengths

#sort by that column
babynames_by_length = babynames.sort_values(by = "name_lengths")

#drop that column
babynames_by_length = babynames_by_length.drop("name_lengths", 1)
babynames_by_length.head(5)

,State,Sex,Year,Name,Count
85197,CA,F,1980,Ai,6
95798,CA,F,1984,Ly,8
309713,CA,M,1989,Ed,5
334020,CA,M,1999,Jr,9
339817,CA,M,2001,Ed,5


Approach 2: Generate an index that is in the order we want. Pass that index to loc.

In [40]:
babynames.loc[babynames["Name"].str.len().sort_values().index].head(5)

,State,Sex,Year,Name,Count,name_lengths
85197,CA,F,1980,Ai,6,2
95798,CA,F,1984,Ly,8,2
309713,CA,M,1989,Ed,5,2
334020,CA,M,1999,Jr,9,2
339817,CA,M,2001,Ed,5,2


How does this work exactly? Let's break it into pieces.

In [41]:
lengths_sorted_by_length = babynames["Name"].str.len().sort_values()
lengths_sorted_by_length.head(5)

85197     2
95798     2
309713    2
334020    2
339817    2
Name: Name, dtype: int64

In [42]:
index_sorted_by_length = lengths_sorted_by_length.index
index_sorted_by_length

Int64Index([ 85197,  95798, 309713, 334020, 339817, 349414, 311134, 130087,
            235709,  31283,
            ...
            314479, 337003, 314358, 102490, 329864, 320036, 326729, 332035,
            319923, 309451],
           dtype='int64', length=394179)

In [43]:
# now pass the index to loc. This is yet another way 
# that loc can be used that we did not discuss in lecture 5.
babynames.loc[index_sorted_by_length].head(5)

,State,Sex,Year,Name,Count,name_lengths
85197,CA,F,1980,Ai,6,2
95798,CA,F,1984,Ly,8,2
309713,CA,M,1989,Ed,5,2
334020,CA,M,1999,Jr,9,2
339817,CA,M,2001,Ed,5,2


Goal 4: Name whose popularity has changed the most. Also tough.

First we need to define change in popularity. For simplicity, for this problem, let's define it as the difference between a name's maximum occurrence and minimum occurrence. For example, let's consider the name Jennifer.

In [44]:
jennifer_counts = babynames.query("Name == 'Jennifer'")["Count"]
jennifer_counts.head(5)

13609     5
16324     5
16992     6
17532    13
18209    24
Name: Count, dtype: int64

In [45]:
max(jennifer_counts) - min(jennifer_counts)

6059

In [46]:
def domain_size(series):
    return max(series) - min(series)

In [47]:
domain_size(jennifer_counts)

6059

In [48]:
name_domain_sizes = babynames.groupby("Name").agg(domain_size)
name_domain_sizes.head(5)

,Year,Count,name_lengths
Name,,,
Aadan,6,2,0
Aadarsh,0,0,0
Aaden,13,148,0
Aadhav,5,2,0
Aadhira,3,4,0


In [49]:
name_domain_sizes.sort_values("Count", ascending=False)

,Year,Count,name_lengths
Name,,,
Michael,110,8254,0
David,110,8111,0
John,110,7320,0
Robert,110,7094,0
Jessica,83,6946,0
...,...,...,...
Halsey,0,0,0
Halston,0,0,0
Halyn,2,0,0


These goals are hard with our tools so far. Will discuss next ime.